In [7]:
import pandas_datareader as web

stocks = []
f = open('data/symbols.txt', 'r')
for line in f:
    stocks.append(line.strip())
    
f.close()
    
web.DataReader(stocks, 'yahoo', start='2000-1-1', end='2019-12-31')['Adj Close'].to_csv('data/prices.csv')
web.DataReader(stocks, 'yahoo', start='2000-1-1', end='2019-12-31')['Volume'].to_csv('data/volume.csv')

In [8]:
import pandas as pd
import numpy as np
import datetime as dt
import math
import warnings

warnings.filterwarnings('ignore')

In [12]:
prices = pd.read_csv('data/prices.csv', index_col='Date', parse_dates=True)
volumechanges = pd.read_csv('data/volume.csv', index_col='Date', parse_dates=True).pct_change()*100

today = dt.date(2000,1,15)
simend = dt.date(2019, 12, 31)
tickers = []
transactionid = 0
money = 1000000
portfolio = {}
activelog = []
transactionlog = []

In [13]:
def getprice(date, ticker):
    global prices
    return prices.loc[date][ticker]

In [14]:
def transaction(id, ticker, amount, price, type, info):
    global transactionid
    if type == "buy":
        exp_date = today + dt.timedelta(days=14)
        transactionid += 1
    else:
        exp_date = today
    if type == "sell":
        data = {"id": id, "ticker": ticker, "amount": amount, "price": price, "date": today, "type": type,
                "exp_date": exp_date, "info": info}
    elif type == "buy":
        data = {"id": transactionid, "ticker": ticker, "amount": amount, "price": price, "date": today, "type": type,
                "exp_date": exp_date, "info": info}
        activelog.append(data)
    transactionlog.append(data)

In [15]:
def buy(interestlst, allocated_money):
    global money, portfolio
    for item in interestlst:
        price = getprice(today, item)
        if not np.isnan(price):
            quantity = math.floor(allocated_money/price)
            money -= quantity*price
            portfolio[item] += quantity
            transaction(0, item, quantity, price, "buy", "")

In [16]:
def sell():
    global money, portfolio, prices, today
    itemstoremove = []
    for i in range(len(activelog)):
        log = activelog[i]
        if log["exp_date"] <= today and log["type"] == "buy":
            tickprice = getprice(today, log["ticker"])
            if not np.isnan(tickprice):
                money += log["amount"]*tickprice
                portfolio[log["ticker"]] -= log["amount"]
                transaction(log["id"], log["ticker"], log["amount"], tickprice, "sell", log["info"])
                itemstoremove.append(i)
            else:
                log["exp_date"] += dt.timedelta(days=1)
    itemstoremove.reverse()
    for elem in itemstoremove:
        activelog.remove(activelog[elem])

In [17]:
def simulation():
    global today, volumechanges, money
    start_date = today - dt.timedelta(days=14)
    series = volumechanges.loc[start_date:today].mean()
    interestlst = series[series > 100].index.tolist()
    sell()
    if len(interestlst) > 0:
        #moneyToAllocate = 500000/len(interestlst)
        moneyToAllocate = currentvalue()/(2*len(interestlst))
        buy(interestlst, moneyToAllocate)

In [22]:
def getindices():
    global tickers
    f = open("data/symbols.txt", "r")
    for line in f:
        tickers.append(line.strip())
    f.close()

In [23]:
def tradingday():
    global prices, today
    return np.datetime64(today) in list(prices.index.values)

In [24]:
def currentvalue():
    global money, portfolio, today, prices
    value = money
    for ticker in tickers:
        tickprice = getprice(today, ticker)
        if not np.isnan(tickprice):
            value += portfolio[ticker]*tickprice
    return int(value*100)/100

In [25]:
def main():
    global today
    getindices()
    for ticker in tickers:
        portfolio[ticker] = 0
    while today < simend:
        while not tradingday():
            today += dt.timedelta(days=1)
        simulation()
        currentpvalue = currentvalue()
        print(currentpvalue, today)
        today += dt.timedelta(days=7)

main()

1000000.0 2000-01-18
1000000.0 2000-01-25
1000000.0 2000-02-01
1000000.0 2000-02-08
1000000.0 2000-02-15
1000000.0 2000-02-22
1000000.0 2000-02-29
1000000.0 2000-03-07
1000000.0 2000-03-14
1000000.0 2000-03-21
1000000.0 2000-03-28
1000000.0 2000-04-04
1000000.0 2000-04-11
1000000.0 2000-04-18
1000000.0 2000-04-25
1000000.0 2000-05-02
1000000.0 2000-05-09
1000000.0 2000-05-16
1000000.0 2000-05-23
1000000.0 2000-05-30
1000000.0 2000-06-06
1000000.0 2000-06-13
1000000.0 2000-06-20
1000000.0 2000-06-27
1000000.0 2000-07-05
1000000.0 2000-07-12
1000000.0 2000-07-19
1000000.0 2000-07-26
1000000.0 2000-08-02
1000000.0 2000-08-09
1000000.0 2000-08-16
1000000.0 2000-08-23
1000000.0 2000-08-30
1000000.0 2000-09-06
1000000.0 2000-09-13
1000000.0 2000-09-20
1000000.0 2000-09-27
1000000.0 2000-10-04
1000000.0 2000-10-11
1000000.0 2000-10-18
1000000.0 2000-10-25
1000000.0 2000-11-01
1000000.0 2000-11-08
1000000.0 2000-11-15
1000000.0 2000-11-22
916666.67 2000-11-29
889743.81 2000-12-06
889743.81 200

1336705.16 2008-01-18
1336705.16 2008-01-25
1336705.16 2008-02-01
1336705.16 2008-02-08
1336705.16 2008-02-15
1336705.16 2008-02-22
1336705.16 2008-02-29
1336705.16 2008-03-07
1336705.16 2008-03-14
1336705.16 2008-03-24
1336705.16 2008-03-31
1336705.16 2008-04-07
1336705.16 2008-04-14
1336705.16 2008-04-21
1336705.16 2008-04-28
1336705.16 2008-05-05
1336705.16 2008-05-12
1336705.16 2008-05-19
1336705.16 2008-05-27
1336705.16 2008-06-03
1336705.16 2008-06-10
1336705.16 2008-06-17
1336705.16 2008-06-24
1336705.16 2008-07-01
1336705.16 2008-07-08
1336705.16 2008-07-15
1336705.16 2008-07-22
1336705.16 2008-07-29
1336705.16 2008-08-05
1336705.16 2008-08-12
1336705.16 2008-08-19
1336705.16 2008-08-26
1336705.16 2008-09-02
1336705.16 2008-09-09
1336705.16 2008-09-16
1336705.16 2008-09-23
1344725.82 2008-09-30
1196623.47 2008-10-07
1202940.24 2008-10-14
1202940.24 2008-10-21
1202940.24 2008-10-28
1202940.24 2008-11-04
1202940.24 2008-11-11
1202940.24 2008-11-18
1202940.24 2008-11-25
1202940.24

1795008.78 2015-07-07
1795008.78 2015-07-14
1795008.78 2015-07-21
1795008.78 2015-07-28
1795008.78 2015-08-04
1795008.78 2015-08-11
1795008.78 2015-08-18
1795008.78 2015-08-25
1795008.78 2015-09-01
1795008.78 2015-09-08
1795008.78 2015-09-15
1795008.78 2015-09-22
1795008.78 2015-09-29
1795008.78 2015-10-06
1795008.78 2015-10-13
1795008.78 2015-10-20
1868309.72 2015-10-27
2089984.47 2015-11-03
2177076.51 2015-11-10
2231011.45 2015-11-17
1922585.51 2015-11-24
1922585.51 2015-12-01
1882531.71 2015-12-08
1802423.99 2015-12-15
1756362.07 2015-12-22
1830461.68 2015-12-29
1782397.09 2016-01-05
1702289.49 2016-01-12
1682262.47 2016-01-19
1672249.01 2016-01-26
1722316.39 2016-02-02
1554090.28 2016-02-09
1562100.94 2016-02-16
1784399.76 2016-02-23
1782397.09 2016-03-01
1742343.29 2016-03-08
1886537.05 2016-03-15
2022720.14 2016-03-22
1956631.32 2016-03-29
2046752.37 2016-04-05
2283070.09 2016-04-12
2457304.32 2016-04-19
2611511.65 2016-04-26
2339145.46 2016-05-03
2287075.42 2016-05-10
2247021.62